# 해외(OECD) 국가들의 사고율, 사망율 비교
1. 국가별 인구수와 사고건수
    - 대한민국은 인구수와 사고건수의 회귀식을 웃도는 사고건수를 가짐
2. 국가별 자동차등록대수와 사고건수
    - 대한민국은 자동차등록대수와 사고건수의 회귀식을 웃도는 사고건수를 가짐
3. 사고건수 대비 치사율
    - 프랑스가 월등히 높고 대한민국은 중간쯤 위치
4. 국가별 인구수와 인구10만명당 교통사고 사망자수 비교
    - 대한민국은 인구수 대비 사망자수가 높음
5. 국가별 사고건수와 인구10만명당 교통사고 사망자수 비교
    - 대한민국은 사고건수 대비 사망자수 높음
6. 국가별 자동차등록대수와 인구10만명당 교통사고 사망자수 비교(scatterplot)
    - 대한민국은 자동차등록대수 대비 교통사고 사망자수가 높음
7. 인구 10만명당 교통사고 사망자수에 영향을 미치는 상관관계 히트맵
    - 상관계수의 절대값이 0.3을 넘어가는 정도의 상관관계 요소 확인하지 못함.
8. 국가별 면허취득교육시간과 인구10만명당사망자수 비교
    - 상관계수 -0.12로 상관관계가 있다고 보기 어려움
9. 국가별 면허취득교육시간과 사고건수(1,000명당 사고건수) 비교
    - 상관계수 0.08로 상관관계가 있다고 보기 어려움

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
plt.rcParams['font.family'] = 'AppleGothic'

In [3]:
# 경고문 끄기
import warnings
warnings.filterwarnings('ignore')

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# OECD 국가들의 사고율, 사망율 비교
- 사고 건수 등 필요값에 결측치가 있는 국가들은 제거

In [9]:
df = pd.read_excel('2018_해외국가사고율비교/OECD국가교통사고비교(2018년).xls', header=1)

FileNotFoundError: ignored

## 데이터 전처리

In [ ]:
df.info()

In [ ]:
df['인구수'] = df['인구수(천명)'] * 1000
df['자동차등록대수'] = df['자동차등록대수(천대)'] * 1000

In [ ]:
df.info()

### G20 국가만 걸러내기 ->표본데이터가 너무 적어지므로 걸러내지 않음

In [ ]:
G20_list = ['미국', '캐나다', '사우디아라비아', '호주', '러시아', '인도', '터키', '남아프리카공화국', '브라질', '아르헨티나', '멕시코', '영국', '프랑스', '독일', '이탈리아', '대한민국', '일본', '중국', '인도네시아']
OECD_list = list(df['국가'].values)

# 비교국가 리스트(G20_list와 OECD_list의 교집합)
com_nation_list = list(set(G20_list)&set(OECD_list))
print(com_nation_list)

In [ ]:
mask = []
for i in df['국가']:
    mask.append(i in com_nation_list)

In [ ]:
mask

In [ ]:
df_com = df[mask]

In [ ]:
df_com

In [ ]:
df = df.replace('-', np.nan)

In [ ]:
df.isnull().sum()

In [ ]:
df['사고건수'].dropna()

In [ ]:
df = df.dropna(subset=['사고건수'])

In [ ]:
df['자동차등록대수'] = df['자동차등록대수(천대)'] * 1000

In [ ]:
df.reset_index(inplace = True, drop=True)

In [ ]:
df_result = df[['국가', '인구수', '자동차등록대수', '사고건수', '사망자수(명)', '인구10만명당사망자수', '자동차1만대당사망자수']]

In [ ]:
df_result

### 숫자형 데이터들의 형 변환

In [ ]:
# 결측치 행 제거
df_result = df_result.dropna()

In [ ]:
df_list = ['인구수', '자동차등록대수', '사고건수', '사망자수(명)']
for col in df_list:
    df_result[col] = df_result[col].astype('int')

In [ ]:
df_result.reset_index(drop=True, inplace=True)

In [ ]:
df_result

In [ ]:
df_ = df_result.sort_values(by='인구10만명당사망자수')

### 단순 사고 건수 비교

In [ ]:
plt.figure(figsize=(12,8))
reg = sns.regplot(x='인구수', y='사고건수', data=df_result, color='blue')
for n in range(len(df_result)):
    if df_result['인구수'][n] > 20000000:
        if df_result['국가'][n] == '대한민국':
            plt.text(df_result['인구수'][n]*1.02, df_result['사고건수'][n]*0.98,
                     df_result['국가'][n], fontsize=13, color='red')
        else:
            plt.text(df_result['인구수'][n]*1.02, df_result['사고건수'][n]*0.98,
                 df_result['국가'][n], fontsize=10)

# 회귀식
z=np.polyfit(df_result['인구수'], df_result['사고건수'], 1) # 기울기와 절편 확인
f=np.poly1d(z)

# 화살표 # 대한민국의 정수형 위치 인덱스가 9이므로 정수형 위치 인덱스 사용함
plt.annotate('',
             xy = (df_result['인구수'][9], df_result['사고건수'][9]), #머리지점
             xytext = (df_result['인구수'][9], f(df_result['인구수'][9])), #꼬리지점
             xycoords = 'data',
             arrowprops = dict(arrowstyle = '->', color = 'red', lw = 3))
plt.title('국가별 인구수와 사고건수', fontsize=20, pad=15)
plt.ylabel('사고건수', fontsize=15)
plt.xlabel('인구수', fontsize=15)

plt.ticklabel_format(style='plain')
# xticks = ['{}'.format(x) + 'K' for x in reg.get_xticks()/1000]
# plt.xticks(xticks)
plt.show()

### 분석결과 : 인구수와 사고건수는 상관 관계를 가진다,
- 대한민국은 그 상관관계를 넘어서는 수준의 인구수 대비 사고 건수가 나타난다.
- 인사이트 풀어서쓰기

# 자동차등록대수와 사고건수의 상관관계

In [ ]:
plt.figure(figsize=(12,8))
sns.regplot(x='자동차등록대수', y='사고건수', data=df_result, color='blue')
for n in range(len(df_result)):
    if df_result['자동차등록대수'][n] > 20000000:
        if df_result['국가'][n] == '대한민국':
            plt.text(df_result['자동차등록대수'][n]*1.02, df_result['사고건수'][n]*0.98,
                     df_result['국가'][n], fontsize=13, color='red')
        else:
            plt.text(df_result['자동차등록대수'][n]*1.02, df_result['사고건수'][n]*0.98,
                     df_result['국가'][n], fontsize=10)

# 회귀식
z=np.polyfit(df_result['자동차등록대수'], df_result['사고건수'], 1) # 기울기와 절편 확인
f=np.poly1d(z)

# 화살표 # 대한민국의 정수형 위치 인덱스가 9이므로 정수형 위치 인덱스 사용함
plt.annotate('',
             xy = (df_result['자동차등록대수'][9], df_result['사고건수'][9]), #머리지점
             xytext = (df_result['자동차등록대수'][9], f(df_result['자동차등록대수'][9])), #꼬리지점
             xycoords = 'data',
             arrowprops = dict(arrowstyle = '->', color = 'red', lw = 3))
plt.title('자동차등록대수와 사고건수의 상관관계', fontsize=20, pad=15)
plt.xlabel('자동차등록대수', fontsize=15)
plt.ylabel('사고건수', fontsize=15)
plt.ticklabel_format(style='plain')
plt.show()

## 사고건수 대비 사망자수(치사율)

In [ ]:
df_result

In [ ]:
df_result['치사율'] = df_result['사망자수(명)'] / df_result['사고건수'] * 100

In [ ]:
df_result

In [ ]:
df_deathRate = df_result.set_index('국가')

In [ ]:
plt.figure(figsize=(12, 8))
sns.barplot(x ='치사율', y= df_deathRate.index, data=df_deathRate.sort_values(by='치사율'), lw=1)
plt.title('사고건수 대비 치사율', fontsize=20, pad=15)
# 텍스트 작성해보기
# for n in range(len(df_deathRate)):
#     plt.text(x=df_deathRate['치사율'][n]*1.02, y=df_deathRate.index[n],
#              s=df_deathRate['치사율'][n], fontsize=13)
plt.xlabel('치사율', fontsize=15)
plt.ylabel('국가명', fontsize=15)
plt.xticks(range(0, 10, 1))
plt.show()

### 강사님 피드백
- 인구 10만명당 사망자 데이터 다시 그려보기 -> 인사이트 추출
    - 이유? -> 교통체계, 과속, 등등...
- 특정도시 기준(서울, 런던 비교 등 해외 특정 도시와 비교)
    - 인구대비 사고가 유의한가? 회귀식
- 사고가 왜 높은가에 대한 탐색

In [ ]:
df_deathRate

In [ ]:
plt.figure(figsize=(12, 8))
sns.scatterplot(x='인구수', y='인구10만명당사망자수', data=df_deathRate, hue='사고건수', size='인구수')
for n in range(len(df_deathRate)):
    if df_deathRate.index[n] == '대한민국':
        plt.text(x=df_deathRate['인구수'][n]*1.02, y=df_deathRate['인구10만명당사망자수'][n]*0.995,
                 s=df_deathRate.index[n] + '(' + str(df_deathRate['인구10만명당사망자수'][n]) +'명' + ')', fontsize=13, color = 'red')
    elif df_deathRate.index[n] == '아이슬란드':
        plt.text(x=df_deathRate['사고건수'][n]*0.99, y=df_deathRate['인구10만명당사망자수'][n]*1.025,
                 s=df_deathRate.index[n] + '(' + str(df_deathRate['인구10만명당사망자수'][n]) +'명' + ')', fontsize=11)
    else:
        plt.text(x=df_deathRate['인구수'][n]*1.02, y=df_deathRate['인구10만명당사망자수'][n]*0.995,
                s=df_deathRate.index[n] + '(' + str(df_deathRate['인구10만명당사망자수'][n]) +'명' + ')', fontsize=11)
plt.title('교통사고 사망율(OECD국가 중 19개국)', fontsize=20, pad=15)
plt.ylabel('인구 10만명당 사망자수', fontsize=15)
plt.xlabel('인구수', fontsize=15)
plt.show()

### 분석결과
- 독일, 일본, 영국은 인구수 대비 교통사고사망율(인구10만명당 사망자수) 낮다.
- 또한 독일, 일본은 사고건수가 많음에도 사망율이 현저히 적다
- 대한민국은 인구수 대비 사망율이 높은 편이며 사고건수 또한 높다.
- 뉴질랜드와 폴란드는 사고건수도 적고 인구수도 적음에도 10만명당 사망자 수가 높은 편이다.
    - 따라서, 인구수와 사고건수 외에 사망율에 영향을 미치는 변수가 존재함을 알 수 있다

- 인구수가 2,000만명 이상인 경우 인구수와 사망율은 음의 상관관계를 그린다.

In [ ]:
plt.figure(figsize=(12, 8))
sns.scatterplot(x='사고건수', y='인구10만명당사망자수', data=df_deathRate, hue='인구수', size='인구수')
for n in range(len(df_deathRate)):
    if df_deathRate.index[n] == '대한민국':
        plt.text(x=df_deathRate['사고건수'][n]*1.02, y=df_deathRate['인구10만명당사망자수'][n]*0.995,
                 s=df_deathRate.index[n] + '(' + str(df_deathRate['인구10만명당사망자수'][n]) +'명' + ')', fontsize=13, color = 'red')
    elif df_deathRate.index[n] == '아이슬란드':
        plt.text(x=df_deathRate['사고건수'][n]*0.99, y=df_deathRate['인구10만명당사망자수'][n]*1.025,
                 s=df_deathRate.index[n] + '(' + str(df_deathRate['인구10만명당사망자수'][n]) +'명' + ')', fontsize=11)
    else:
        plt.text(x=df_deathRate['사고건수'][n]+1.02, y=df_deathRate['인구10만명당사망자수'][n]*0.995,
                 s=df_deathRate.index[n] + '(' + str(df_deathRate['인구10만명당사망자수'][n]) +'명' + ')', fontsize=11)
plt.title('사고건수 대비 교통사고 사망율(OECD국가 중 19개국)', fontsize=20)
plt.ylabel('인구 10만명당 사망자수', fontsize=18)
plt.xlabel('사고건수', fontsize=18)
plt.show()

### 분석결과
- 독일, 일본은 사고건수는 많음에도 사망율이 적다
- 스위스, 영국, 스웨덴, 일본은 사망율이 가장 적은 4개국. -> 영국, 일본은 좌측통행
- 뉴질랜드, 폴란드, 대한민국은 사망율이 가장 높은 3개국이다. -> 뉴질랜드도 좌측통행

- 대한민국은 사고건수도 많고, 사망율도 높다 -> 대책이 필요하다
    - 사고건수가 높으나 사망율이 낮은 독일,일본 그리고 사고 건수가 낮으나 사망율이 높은 뉴질랜드, 폴란드를 비교할 필요가 있어보인다
    - 지역적 특성, 병원, 인구수,
    - __인구수는 독일, 일본은 많고, 뉴질랜드, 폴란드는 적으며 대한민국은 그 중간__
        -
- 탐색 필요한 것들?
    - 면적, 면적대비 인구수
    - 면허취득, 국제운전면허증 사용 가능 여부
    - 운전 수단
    - 차종...?
        - 기업적으로 잘팔리는 차종 혹은 자주 타고다니는 차종(이륜차, 승용차, 승합차 등등)
        - 독삼사, 일제차, 영국
    - 기상여건
    - 병원, 응급의료시설
    - 과속에 대한 문제, 시내 혹은 고속도로의 평균 제한 속도
    - 교통법규 위반건수(범칙금 데이터 등) ==> 범칙금 체계, 3아웃체제, 법집행에 대한 사회적, 행정적 태도
    - 해외의 고령운전자 비율
    -

#### 히트맵 뽑아보기


In [ ]:
plt.figure(figsize=(12, 8))
sns.scatterplot(x='자동차등록대수', y='인구10만명당사망자수', data=df_deathRate, hue='인구수', size='인구수')
for n in range(len(df_deathRate)):
    if df_deathRate.index[n] == '대한민국':
        plt.text(x=df_deathRate['자동차등록대수'][n]*1.02, y=df_deathRate['인구10만명당사망자수'][n]*0.995,
                 s=df_deathRate.index[n] + '(' + str(df_deathRate['인구10만명당사망자수'][n]) +'명' + ')', fontsize=13, color = 'red')
    else:
        plt.text(x=df_deathRate['자동차등록대수'][n]*1.02, y=df_deathRate['인구10만명당사망자수'][n]*0.995,
                 s=df_deathRate.index[n] + '(' + str(df_deathRate['인구10만명당사망자수'][n]) +'명' + ')', fontsize=11)
plt.title('자동차등록대수 대비 교통사고 사망율(OECD국가 중 19개국)', fontsize=20, pad=15)
plt.ylabel('인구 10만명당 사망자수', fontsize=15)
plt.xlabel('자동차등록대수', fontsize=15)
plt.ticklabel_format(style='plain')
plt.show()

### 분석결과
- 자동차 등록대수 대비 높은 사망율 : 뉴질랜드, 폴란드, 대한민국
- 중간 : 프랑스, 이탈리아, 스페인
- 자동차 등록대수 대비 낮은 사망율 : 일본, 독일, 영국

In [ ]:
df_deathRate

# 인구10만명당 사망자수에 영향을 미치는 변수 파악하기

In [ ]:
df_corr = df_deathRate.corr().loc[:'인구10만명당사망자수', ['인구수', '자동차등록대수', '사고건수', '사망자수(명)', '인구10만명당사망자수']]

In [ ]:
df_corr_origin =  df_deathRate.corr().loc[:'인구10만명당사망자수', ['인구수', '자동차등록대수', '사고건수',  '사망자수(명)', '인구10만명당사망자수']]

In [ ]:
df_corr

# 히트맵 시각화

In [ ]:
# mask만들기
mask = np.zeros_like(df_corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

plt.figure(figsize=(16,12))

sns.heatmap(df_corr_origin, annot=True, mask=mask, cmap='Reds', linewidths=.5, fmt= '.2f', annot_kws={'size': 15})
plt.title('인구10만명당 사망자수와 다른 변수의 상관관계')
plt.show()

# 분석결과
- 인구 10만명당 사망자수와 다른 변수들의 상관관계를 살펴보니, 모두 절대값 0.3이하로 상관관계가 있다고 보기 어렵다.
- 인구 10만명당 사망자수에 영향을 미치는 다른 변수가 있을 것으로 판단된다.

In [ ]:
df_deathRate

In [ ]:
df_deathRate.index

## 면허취득시 필요한 교육 시간과 인구10만명당사망자수의 상관관계

In [ ]:
# 빈 컬럼 추가
df_deathRate['면허취득교육시간'] = pd.Series()

In [ ]:
# 키=국가명, 값=교육시간으로 하는 dictionary 만들어서 반복문으로 컬럽에 값 추가
nat_edutime_dict = {'대한민국': 13,
                    '폴란드': 60,
                    '프랑스': 20,
                    '이탈리아': 6,
                    '일본': 120,
                    '독일': 34,
                    }

In [ ]:
for nation, time in nat_edutime_dict.items():
    df_deathRate.loc[nation, '면허취득교육시간'] = time

In [ ]:
df_deathRate

In [ ]:
df_deathRate.index

In [ ]:
# 면허 취득 교육시간 값만 있는 국가만 가지고 새로운 df 생성
df_edutime = df_deathRate.loc[nat_edutime_dict.keys(), ]

In [ ]:
df_edutime.sort_values('면허취득교육시간')

In [ ]:
plt.figure(figsize=(12, 8))
sns.scatterplot(x='면허취득교육시간', y='인구10만명당사망자수', data=df_edutime, s=50)
for n in range(len(df_edutime)):
    if df_edutime.index[n] == '대한민국':
        plt.text(df_edutime['면허취득교육시간'][n]+1, df_edutime['인구10만명당사망자수'][n]*0.995,
                 df_edutime.index[n], fontsize=15, color='red')
    else:
        plt.text(df_edutime['면허취득교육시간'][n]+1, df_edutime['인구10만명당사망자수'][n]*0.995,
             df_edutime.index[n], fontsize=12)

# 추세선 시각화 하기 위해 1차식 구함
pf = np.polyfit(df_edutime['면허취득교육시간'], df_edutime['인구10만명당사망자수'], 1)
f1 = np.poly1d(pf)
fx = np.linspace(5, 120, num=100)
plt.plot(fx, f1(fx), ls='dashed', lw=3, color='g')

plt.title('면허취득에 필요한 교육 시간과 사망율의 상관관계', fontsize=20, pad=10)
plt.xlabel('면허취득에 필요한 교육 시간', fontsize=15)
plt.ylabel('인구10만명당사망자수', fontsize=15)
plt.show()

In [ ]:
df_edutime.corr()

## 분석결과
- 면허취득에 필요한 교육 시간과 인구10만명당사망자수의 상관계수는 -0.446으로 음의 상관관계가 있다.
    - 따라서, 면허 취득에 필요한 교육시간이 증가할 수록 인구10만명당사망자수는 감소하는 경향을 보인다.
- 다만, 표본이 6개국밖에 되지 않아 추가 조사가 필요해보이고, 면허취득에 필요한 교육 시간 데이터를 확보하는 데는 다소 어려움이 있을 것으로 예상된다

# 면허 교육 시간과 인구 10만명당 사망자수 다시 그리기
drop으로 잘라내지 않고 10만명당 사망자수와 면허 교육시간만 비교하기

In [ ]:
temp_df = pd.read_excel('./2018_해외국가사고율비교/OECD국가교통사고비교(2018년).xls', header=1)

In [ ]:
temp_df.set_index('국가', inplace=True)

In [ ]:
temp_df.rename(index={'터키': '튀르키예(터키)'}, inplace=True)

In [ ]:
nat_all_edutime_dict = {'대한민국': 13,
                        '폴란드': 60,
                        '프랑스': 20,
                        '이탈리아': 6,
                        '일본': 120,
                        '독일': 34,
                        '헝가리': 30,
                        '오스트리아': 50,
                        '핀란드': 22,
                        '체코': 64,
                        '스위스': 18,
                        '벨기에': 20,
                        '튀르키예(터키)': 42,
                    }

In [ ]:
for nation, time in nat_all_edutime_dict.items():
    temp_df.loc[nation, '면허취득교육시간'] = time

In [ ]:
temp_df

In [ ]:
edu_time_all = temp_df.dropna(axis=0, subset=['면허취득교육시간'])

In [ ]:
plt.figure(figsize=(12, 8))
sns.scatterplot(x='면허취득교육시간', y='인구10만명당사망자수', data=edu_time_all, s=50)
for n in range(len(edu_time_all)):
    if edu_time_all.index[n] == '대한민국':
        plt.text(edu_time_all['면허취득교육시간'][n]+1, edu_time_all['인구10만명당사망자수'][n]*0.995,
                 edu_time_all.index[n], fontsize=15, color='red')
    else:
        plt.text(edu_time_all['면허취득교육시간'][n]+1, edu_time_all['인구10만명당사망자수'][n]*0.995,
                 edu_time_all.index[n], fontsize=12)

# 추세선 시각화 하기 위해 1차식 구함
pf = np.polyfit(edu_time_all['면허취득교육시간'].astype('float'), edu_time_all['인구10만명당사망자수'].astype('float'), 1)
f1 = np.poly1d(pf)
fx = np.linspace(5, 120, num=100)
plt.plot(fx, f1(fx), ls='dashed', lw=3, color='b')

plt.title('면허취득에 필요한 교육 시간과 사망율의 상관관계', fontsize=20, pad=10)
plt.xlabel('면허취득에 필요한 교육 시간', fontsize=15)
plt.ylabel('인구10만명당사망자수', fontsize=15)
plt.xticks(np.arange(0, 131, 10))
plt.yticks(np.arange(3, 9, 0.5))
plt.show()

In [ ]:
np.corrcoef(edu_time_all['면허취득교육시간'].astype('float'), edu_time_all['인구10만명당사망자수'].astype('float'))

### 분석 결과
- 면허 취득 교육 시간과, 인구 10만명당 사망율의 상관계수는 -1.23으로, 상관 관계가 있다고 보기 어렵다.
- 다만, 우리나라의 면허 취득에 필요한 교육시간이 다른 국가들에 비해 현저히 적은 것은 확인된다.
- 위 분석은 면허 시험의 난이도는 측정하지 않고, 단순 교육 시간만 비교한 것이다.


# 사고건수와 면허취득 교육시간 비교
- 사고건수를 인구수로 나워 1천명당 사고건수로 스케일링하기
-

In [ ]:
edu_time_all['1천명 당 사고건수'] = edu_time_all['사고건수'] / edu_time_all['인구수(천명)']

In [ ]:
edu_time_all

In [ ]:
plt.figure(figsize=(12, 8))
sns.scatterplot(x='면허취득교육시간', y='1천명 당 사고건수', data=edu_time_all, s=50, hue='인구수(천명)')
for n in range(len(edu_time_all)):
    if edu_time_all.index[n] == '대한민국':
        plt.text(edu_time_all['면허취득교육시간'][n]+1, edu_time_all['1천명 당 사고건수'][n]*0.995,
                 edu_time_all.index[n], fontsize=15, color='red')
    else:
        plt.text(edu_time_all['면허취득교육시간'][n]+1, edu_time_all['1천명 당 사고건수'][n]*0.995,
                 edu_time_all.index[n], fontsize=12)

# 추세선 시각화 하기 위해 1차식 구함
pf = np.polyfit(edu_time_all['면허취득교육시간'].astype('float'), edu_time_all['1천명 당 사고건수'].astype('float'), 1)
f1 = np.poly1d(pf)
fx = np.linspace(5, 120, num=100)
# plt.plot(fx, f1(fx), ls='dashed', lw=3, color='g') 회귀식을 그래프에 그리는 것이 필요한가 ?

plt.title('면허취득에 필요한 교육 시간과 사고건수의 상관관계', fontsize=20, pad=10)
plt.xlabel('면허취득에 필요한 교육 시간', fontsize=15)
plt.ylabel('1천명당 사고건수', fontsize=15)
# plt.xticks(np.arange(0, 131, 10))
# plt.yticks(np.arange(3, 8, 0.5))
plt.show()

In [ ]:
np.corrcoef(edu_time_all['면허취득교육시간'].astype('float'), edu_time_all['1천명 당 사고건수'].astype('float'))

### 분석결과
- 면허취득에 필요한 교육 시간과 1천명 당 사고 건수의 상관계수는 0.08로 상관관계가 거의 없다.
- 그렇다면, 세계 각국의 교통사고로 인한 사망율(인구10만명당 사망자수)는 다른 복합적인 변수들에 의해 변화한다고 볼 수 있다.